# Entraînement segmentation poisson (U-Net)

Bloc-notes pour préparer les données, vérifier les masques et lancer l'entraînement avec `train_unet.py`.

## 1) Pré-requis
- Python 3.10+ recommandé
- GPU CUDA optionnel mais conseillé
- Données organisées comme :
```
data_root/
  train/images/*.jpg|png
  train/masks/*.png       # masques binaires 0/1
  val/images/*.jpg|png
  val/masks/*.png
```

In [17]:
!pip install torch==2.4.1+cpu torchvision==0.19.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
# 2) Installation des dépendances (exécuter une seule fois)
# Astuce: commente la ligne si tu as déjà installé.
import sys, subprocess, os, json
if os.environ.get("SKIP_PIP_INSTALL") == "1":
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], check=True)

Looking in links: https://download.pytorch.org/whl/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch==2.4.1+cpu (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu113, 1.11.0+cu115, 1.12.0, 1.12.0+cpu, 1.12.0+cu113, 1.12.0+cu116, 1.12.1, 1.12.1+cpu, 1.12.1+cu113, 1.12.1+cu116, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121, 2.1.1, 2.1.1+cpu, 2.1.1+cu118, 2.1.1+cu121, 2.1.2, 2.1.2+cpu, 2.1.2+cu118, 2.1.2+cu121, 2.2.0, 2.2.0+cpu, 2.2.0+cu118, 2.2.0+cu121, 2.2.1, 2.2.1+cpu, 2.2.1+cu118, 2.2.1+cu121, 2.2.2, 2.2.2+cpu, 2.2.2+cu118, 2.2.2+cu121, 2.3.0, 2.3.0+cpu, 2.3.0+cu118, 2.3.0+cu121, 2.3.1, 2.3.1+cpu, 2.3.1+cu118, 2.3.1+cu121, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.4.1+cpu

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe

## 3) Chemins et paramètres

In [18]:
from pathlib import Path

# Chemin racine du dataset
data_root = Path(r"C:\Users\USER\Downloads\Data_Kosmos")
# Paramètres d'entraînement
epochs = 20
batch_size = 4
lr = 1e-3
size = 512
device = "cuda"  # "cpu" si pas de GPU
model_name = "smp"  # "smp" (Unet + encoder) ou "simple" (petit UNet custom)
base_channels = 32
encoder = "resnet34"
use_pretrained = True  # False pour désactiver le pretrained
use_amp = True  # conseillé sur GPU

## 4) Vérification des dossiers et des masques
On vérifie que chaque image a un masque correspondant.

In [20]:
def list_pairs(images_dir: Path, masks_dir: Path):
    images = sorted(images_dir.glob("*"))
    masks = sorted(masks_dir.glob("*"))
    if not images:
        raise FileNotFoundError(f"Aucune image trouvée dans {images_dir}")
    missing = []
    mask_names = {m.name for m in masks}
    for img in images:
        # On cherche un masque avec même nom mais extension libre
        base = img.stem
        candidates = [n for n in mask_names if Path(n).stem == base]
        if not candidates:
            missing.append(img.name)
    return images, masks, missing

train_images, train_masks, train_missing = list_pairs(
    data_root / "train" / "images",
    data_root / "train" / "masks",
)

val_images, val_masks, val_missing = list_pairs(data_root / "val" / "images", data_root / "val" / "masks")

print(f"Train: {len(train_images)} images, {len(train_masks)} masques")
print(f"Val:   {len(val_images)} images, {len(val_masks)} masques")
if train_missing:
    print(f"Manque {len(train_missing)} masques en train (exemple): {train_missing[:5]}")
if val_missing:
    print(f"Manque {len(val_missing)} masques en val (exemple): {val_missing[:5]}")
if train_missing or val_missing:
    raise SystemExit("Compléter les masques avant l'entraînement.")


Train: 5244 images, 0 masques
Val:   937 images, 0 masques
Manque 5244 masques en train (exemple): ['0117_f000000.jpg', '0117_f000024.jpg', '0117_f000048.jpg', '0117_f000072.jpg', '0117_f000096.jpg']
Manque 937 masques en val (exemple): ['0122_f000000.jpg', '0122_f000024.jpg', '0122_f000048.jpg', '0122_f000072.jpg', '0122_f000096.jpg']


SystemExit: Compléter les masques avant l'entraînement.

c:\Users\USER\Downloads\Projet-KOSMOS-dev\.venv\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 5) Visualisation rapide d'une image + masque
(Utile pour vérifier que les masques sont binaires et alignés)

In [ ]:
import matplotlib.pyplot as plt
import cv2

sample_idx = 0
img_path = train_images[sample_idx]
mask_path = (data_root / "train" / "masks" / img_path.name)
image = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(image)
axs[0].set_title("Image")
axs[0].axis("off")

axs[1].imshow(mask, cmap="gray")
axs[1].set_title("Masque")
axs[1].axis("off")

axs[2].imshow(image)
axs[2].imshow(mask, cmap="jet", alpha=0.4)
axs[2].set_title("Overlay")
axs[2].axis("off")
plt.tight_layout()
plt.show()

## 6) Lancer l'entraînement
On réutilise la fonction `train` du script `train_unet.py`.

In [ ]:
from train_unet import train

encoder_weights = "imagenet" if (model_name == "smp" and use_pretrained) else None

train(
    data_root=data_root,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr,
    size=size,
    device=device,
    model_name=model_name,
    base_channels=base_channels,
    encoder=encoder,
    encoder_weights=encoder_weights,
    use_amp=use_amp,
)


## 7) Résultats
- Le meilleur modèle est sauvegardé dans `artifacts/best_unet_fish.pth`.
- Le log de chaque époque affiche la loss et l'IoU validation.
- Ajuste `batch_size` si tu es limité en mémoire GPU/CPU.